# Generating GO Transit Zone Map Using GO-GTFS and Open-Source Tools (Views are my own and do not represent the actual look of GO Transit Fare Zones)

GO Transit utilizes a fare-by-zone system, with fares dependant on the amount of zones a rider passes through; said fares are defined by Metrolinx By-law No. 2A. As of December 2025, GO Transit has 90 active fare zones.

In [1]:
import folium, json, os, shapely, sys
import branca.colormap as cm
import geopandas as gpd
import numpy as np
import pandas as pd
from branca.element import Template, MacroElement
from datetime import datetime
from folium import plugins
from pathlib import Path
from scipy.spatial import Voronoi
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union

if os.path.join("..", "src") not in sys.path:
    sys.path.append(os.path.join("..", "src"))
from gtfs_cleaning import load_gtfs_data_from_zip
from build_voronoi_zones import voronoi_finite_polygons

## Define the relative path of config.json

In [2]:
config_path = Path("../config.json")

# Load the JSON file and resolve paths dynamically based on the location of config.json
with config_path.open() as file:
    config = json.load(file)
config["data_GTFS_folder"] = str((config_path.parent / config["data_GTFS_folder"]).resolve())
config["data_GIS_folder"] = str((config_path.parent / config["data_GIS_folder"]).resolve())
config["outputs_folder"] = str((config_path.parent / config["outputs_folder"]).resolve())

## Data Preparation

### Load specified GO-GTFS files

In [3]:
# If you want to load all the data, make gtfs_to_load = None.
gtfs_to_load = ['stops', 'stop_times', 'trips', 'routes',
                'fare_attributes', 'fare_rules', 'shapes'] 

gtfs_zip_path = os.path.join(config['data_GTFS_folder'], 'gtfs.zip')

gtfs_data = load_gtfs_data_from_zip(gtfs_zip_path, gtfs_to_load)

# Promote each loaded DataFrame to a variable named after the key
for item in gtfs_to_load:
    df = gtfs_data.get(item)
    if df is None:
        print(f"{item}: Not loaded or missing in gtfs_data — skipping")
        continue
    globals()[item] = df
    print(f"{item.capitalize()}: {df.shape}")


# print("Number of unique zones:", stops.zone_id.nunique())

Loading stops.txt from zip...
Loaded stops.txt with 918 rows.
Finished processing stops.txt.
Loading stop_times.txt from zip...
Loaded stop_times.txt with 3418130 rows.
Finished processing stop_times.txt.
Loading trips.txt from zip...
Loaded trips.txt with 199848 rows.
Finished processing trips.txt.
Loading routes.txt from zip...
Loaded routes.txt with 88 rows.
Finished processing routes.txt.
Loading fare_attributes.txt from zip...
Loaded fare_attributes.txt with 8100 rows.
Finished processing fare_attributes.txt.
Loading fare_rules.txt from zip...
Loaded fare_rules.txt with 8100 rows.
Finished processing fare_rules.txt.
Loading shapes.txt from zip...
Loaded shapes.txt with 729622 rows.
Finished processing shapes.txt.
Stops: (918, 5)
Stop_times: (3418130, 6)
Trips: (199848, 7)
Routes: (88, 5)
Fare_attributes: (8100, 2)
Fare_rules: (8100, 3)
Shapes: (729622, 4)


### Load GeoDataFrame of stops and GO Transit Zone Boundary
GO Transit Boundary is a bounding box which includes all GO Transit stops inside. It does not reflect any actual files exist within Metrolinx.

In [4]:
# Create GeoDataFrame of stops
gdf = gpd.GeoDataFrame(
    stops,
    geometry=[Point(xy) for xy in zip(stops["stop_lon"], stops["stop_lat"])],
    crs="EPSG:4326"
)
# Project to UTM 17N for robust geometry ops
gdf_utm = gdf.to_crs("EPSG:26917")

# Load GO transit zone boundary, then ensure boundary is in WGS84 and dissolve/union to a single polygon
boundary_gdf = gpd.read_file(os.path.join(config['data_GIS_folder'], "GO_zone.geojson"))
boundary_ll = boundary_gdf.to_crs("EPSG:4326")
# Use boundary_ll as the clipping geometry (lat/lon)
boundary_union_ll = unary_union(boundary_ll.geometry)
# Project boundary to UTM as well
boundary_utm = gpd.GeoSeries([boundary_union_ll], crs="EPSG:4326").to_crs("EPSG:26917")[0]

## Generating Voronoi polygons

In [5]:
# Points in UTM
coords = np.array([(geom.x, geom.y) for geom in gdf_utm.geometry])

# Get unique coordinates and their indices 
_, idx_unique = np.unique(coords, axis=0, return_index=True)
# De-duplicate identical points (SciPy Voronoi requires uniqueness)
if len(idx_unique) < len(coords):
    rng = np.random.default_rng(42) # fixed seed for reproducibility
    dup_mask = np.ones(len(coords), dtype=bool); dup_mask[idx_unique] = False # de-duplication mask
    coords[dup_mask] += rng.normal(scale=1.0, size=(dup_mask.sum(), 2))  # ~1 m

vor = Voronoi(coords)

regions, vertices = voronoi_finite_polygons(vor, radius=None)

# Clean boundary first
boundary_utm = gpd.GeoSeries([boundary_union_ll], crs="EPSG:4326").to_crs("EPSG:26917")
boundary_utm = boundary_utm.buffer(0).union_all()  # fix self-intersections

# Create polygons and clip to boundary
polys = []
for region in regions:
    poly = shapely.Polygon(vertices[region])
    clipped = poly.intersection(boundary_utm)
    if not clipped.is_valid:
        clipped = clipped.buffer(0)
    polys.append(clipped)

# Create GeoDataFrame of Voronoi cells, to be dissolved later
cells_gdf = gpd.GeoDataFrame(gdf_utm[["stop_id","zone_id"]].copy(),
                             geometry=polys, crs="EPSG:26917")

from shapely.ops import snap, unary_union
# Snap cells very slightly to the boundary to align edges (tolerance in meters)
cells_gdf["geometry"] = cells_gdf["geometry"].apply(lambda g: snap(g, boundary_utm, 0.5))  # 0.5 m
cells_gdf["geometry"] = cells_gdf.buffer(0)

zones_gdf_utm = cells_gdf.dissolve(by="zone_id", as_index=False)
zones_gdf_utm["geometry"] = zones_gdf_utm.buffer(0)

mask_union = unary_union(cells_gdf.geometry)
residual = boundary_utm.difference(mask_union)

if not residual.is_empty:
    # Convert residual to polygons
    res_polys = [residual] if residual.geom_type != "MultiPolygon" else list(residual.geoms)
    residual_gdf = gpd.GeoDataFrame(geometry=res_polys, crs="EPSG:26917")

    # Assign to nearest stop's zone
    # Use centroids for the residual pieces
    residual_centroids = residual_gdf.copy()
    residual_centroids["geometry"] = residual_centroids.centroid
    nearest = gpd.sjoin_nearest(residual_centroids, gdf_utm[["zone_id","geometry"]], how="left", distance_col="dist")
    residual_gdf["zone_id"] = nearest["zone_id"].values

    # Merge back and re-dissolve
    cells_gdf = pd.concat([cells_gdf, residual_gdf], ignore_index=True)
    cells_gdf["geometry"] = cells_gdf.buffer(0)
    zones_gdf_utm = cells_gdf.dissolve(by="zone_id", as_index=False).buffer(0)

In [6]:
# Keep only relevant columns
zones_gdf_utm = zones_gdf_utm.drop(
    columns=[c for c in zones_gdf_utm.columns if c not in ['zone_id', 'geometry']]
)

# Reproject back to WGS84 before saving as geojson
cells_gdf_ll = cells_gdf.to_crs("EPSG:4326")
zones_gdf = zones_gdf_utm.to_crs("EPSG:4326")

# Save Voronoi cells and zones as GeoJSON
cells_gdf_ll.to_file(os.path.join(config["data_GIS_folder"], "go_transit_voronoi_cells.geojson"), 
                     driver="GeoJSON")
zones_gdf.to_file(os.path.join(config["data_GIS_folder"], "go_transit_voronoi_zones.geojson"), 
                      driver="GeoJSON")

## Generate html go transit voronoi fare zone map

In [14]:
# Rename gdf back to stops_ll for mapping
stops_ll = gdf

# Build a Folium map
center_lat = stops_ll["stop_lat"].mean()
center_lon = stops_ll["stop_lon"].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=9, tiles="CartoDB positron")

unique_zones = sorted(zones_gdf["zone_id"].unique())
cmap = cm.linear.Set1_09.scale(0, len(unique_zones))
zone_color = {z: cmap(i) for i, z in enumerate(unique_zones)}

# Function to style zones
def style_fn(feature):
    zid = feature["properties"]["zone_id"]
    return {
        "fillColor": zone_color.get(zid, "#888888"),
        "color": zone_color.get(zid, "#444444"),
        "weight": 1,
        "fillOpacity": 0.35
    }

folium.GeoJson(
    zones_gdf[["zone_id","geometry"]].to_json(),
    name="Zones (Voronoi Polygons)",
    style_function=style_fn,
    tooltip=folium.features.GeoJsonTooltip(fields=["zone_id"], aliases=["Zone ID"])
).add_to(m)

stops_fg = folium.FeatureGroup(name="Stops")
for _, row in stops_ll.iterrows():
    folium.CircleMarker(
        location=[row["stop_lat"], row["stop_lon"]],
        radius=4,
        color="#2b8cbe",
        fill=True,
        fill_opacity=0.9,
        popup=f"{row['stop_name']} (zone {row['zone_id']})"
    ).add_to(stops_fg)
stops_fg.add_to(m)

# Add the boundary to the map
folium.GeoJson(
    boundary_gdf.to_json(),
    style_function=lambda feature: {
        'fillColor': 'none',  # No fill color
        'color': 'black',     # Border color
        'weight': 2,          # Border thickness
        'opacity': 0.7        # Border opacity
    },
    name='Boundary'
).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

# Add fixed disclaimer panel with a brief method summary
disclaimer = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed;
    bottom: 10px;
    left: 10px;
    z-index: 9999;
    background: rgba(255,255,255,0.95);
    padding: 8px 10px;
    border: 1px solid #bbb;
    border-radius: 6px;
    box-shadow: 0 1px 4px rgba(0,0,0,0.25);
    max-width: 380px;
    font-size: 12px;
    line-height: 1.35;">
        
    <div style="font-weight:600; margin-bottom:4px;">Disclaimer</div>
        
    <div>This is an independent, personal project and does not represent official GO Transit fare zones, designs, or policies. All views are solely my own.</div>
        
    <div style="margin-top:6px;">
        The zones shown are derived from publicly available <a href="https://www.gotransit.com/en/partner-with-us/software-developers" target="_blank" rel="noopener">GO GTFS</a> using Voronoi diagram in a projected coordinate reference system. The polygons were clipped to a custom-generated service boundary, designed to ensure all GO Transit stops and routes fall within it, and dissolved by zone identifier. Finally, the zones were reprojected to WGS84 for web visualization.
    </div>
        
    <div style="margin-top:6px;">
        Source code and methodology: <a href="https://github.com/zhenhuasun/go-zone-network-explorer/" target="_blank" rel="noopener">github.com/zhenhuasun/go-zone-network-explorer/</a>
    </div>
</div>
{% endmacro %}
"""

macro = MacroElement()
macro._template = Template(disclaimer)
m.get_root().add_child(macro)

m.save(os.path.join(config["outputs_folder"], 
                    "01_go_transit_voronoi_fare_zones.html"))

## Generate html go transit voronoi fare zone map with treelayercontrol

In [15]:
# Rename gdf back to stops_ll for mapping
stops_ll = gdf

stops_new = stops_ll.iloc[:,:5] \
    .merge(stop_times[['stop_id', 'trip_id']], on='stop_id', how='right') \
    .merge(trips[['trip_id', 'route_id']], on='trip_id', how='left') \
    .merge(routes[['route_id', 'route_short_name', 'route_long_name', 'route_color']], on='route_id', how='left')
stops_new['route_full_name'] = stops_new['route_short_name'] + ": " + stops_new['route_long_name']

stops_new = stops_new.drop_duplicates(['stop_id','route_full_name'])[['stop_id','stop_name','stop_lat','stop_lon','zone_id','route_color', 'route_full_name']]
stops_new["Train"] = stops_new["stop_id"].apply(lambda x: 1 if x.isalpha() and x != "PA" else 0)
stops_new["Bus"] = stops_new["stop_id"].apply(lambda x: 1 if x.isdigit() or x == "PA" else 0)
stops_new['stop_color'] = stops_new.apply(
    lambda row: '000000' if row['stop_id'] == 'UN'
    else '696969' if row['Bus'] == 1
    else row['route_color'],
    axis=1
)
stops_new = stops_new[['stop_id','stop_name','stop_lat','stop_lon','zone_id',
                       'Train','Bus','route_full_name','route_color','stop_color']]
stops_new['route_color'] = stops_new['route_color'].apply(
    lambda x: '; '.join(f'#{c}' if not c.startswith('#') else c for c in x.split('; '))
)
stops_new = stops_new.sort_values(['stop_id','route_full_name'], ascending=True).reset_index(drop=True)

stops_new

,stop_id,stop_name,stop_lat,stop_lon,zone_id,Train,Bus,route_full_name,route_color,stop_color
0,00005,Yonge St. @ Baif Blvd. (Hillcrest Mall),43.855755,-79.433876,50,0,1,61: Richmond Hill,#0099c7,696969
1,00006,Yonge St. @ Hwy. 407,43.831982,-79.428383,60,0,1,32: Brampton Trinity Common / North York,#00853e,696969
2,00008,Yonge St. @ Sheppard Ave.,43.761597,-79.411194,5,0,1,19: Mississauga / North York,#f57f25,696969
3,00008,Yonge St. @ Sheppard Ave.,43.761597,-79.411194,5,0,1,27: Milton / North York,#f57f25,696969
4,00008,Yonge St. @ Sheppard Ave.,43.761597,-79.411194,5,0,1,67: Keswick / North York,#003767,696969
...,...,...,...,...,...,...,...,...,...,...
1674,UN,Union Station GO,43.645195,-79.380600,2,1,0,RH: Richmond Hill,#0099c7,000000
1675,UN,Union Station GO,43.645195,-79.380600,2,1,0,ST: Stouffville,#794500,000000
1676,WE,Weston GO,43.700220,-79.514671,4,1,0,KI: Kitchener,#00853e,00853e
1677,WH,Whitby GO,43.864840,-78.938180,93,1,0,LE: Lakeshore East,#ff0d00,ff0d00


In [16]:
# Step 1: Group and collect (route_full_name, route_color) pairs
grouped = stops_new.groupby('stop_id').agg({
    'stop_name': 'first',
    'stop_lat': 'first',
    'stop_lon': 'first',
    'zone_id': 'first',
    'Train': 'max',
    'Bus': 'max',
    'stop_color': 'first',
    'route_full_name': list,
    'route_color': list
}).reset_index()

# Step 2: Zip full_name and color, drop duplicates while preserving order
def zip_and_clean(names, colors):
    seen = set()
    clean = []
    for name, color in zip(names, colors):
        if name not in seen:
            seen.add(name)
            clean.append((name, color))
    return clean

grouped['route_pairs'] = grouped.apply(
    lambda row: zip_and_clean(row['route_full_name'], row['route_color']), axis=1
)

# Step 3: Separate cleaned tuples into two lists again
grouped['route_full_name'] = grouped['route_pairs'].apply(lambda pairs: '; '.join([p[0] for p in pairs]))
grouped['route_color'] = grouped['route_pairs'].apply(lambda pairs: '; '.join([p[1] for p in pairs]))

# Step 4: Final cleanup
stops_final = grouped.drop(columns=['route_pairs'])

# Create rail_stop_label column
stops_final['rail_stop_label'] = np.select([stops_final['Train'] == 0,stops_final['stop_id'] == 'UN'],
                                           ['Bus','Union Station'],
                                           default=stops_final['route_full_name'])
stops_final

,stop_id,stop_name,stop_lat,stop_lon,zone_id,Train,Bus,stop_color,route_full_name,route_color,rail_stop_label
0,00005,Yonge St. @ Baif Blvd. (Hillcrest Mall),43.855755,-79.433876,50,0,1,696969,61: Richmond Hill,#0099c7,Bus
1,00006,Yonge St. @ Hwy. 407,43.831982,-79.428383,60,0,1,696969,32: Brampton Trinity Common / North York,#00853e,Bus
2,00008,Yonge St. @ Sheppard Ave.,43.761597,-79.411194,5,0,1,696969,19: Mississauga / North York; 27: Milton / Nor...,#f57f25; #f57f25; #003767; #ff0d00,Bus
3,00011,York Mills Bus Terminal,43.745079,-79.406464,5,0,1,696969,33: Guelph / North York; 36: Brampton / North ...,#00853e; #00853e; #ff0d00,Bus
4,00013,Finch Bus Terminal,43.782173,-79.415093,5,0,1,696969,19: Mississauga / North York; 27: Milton / Nor...,#f57f25; #f57f25; #00853e; #003767; #ff0d00,Bus
...,...,...,...,...,...,...,...,...,...,...,...
913,UI,Unionville GO,43.851689,-79.314332,71,1,0,794500,ST: Stouffville,#794500,ST: Stouffville
914,UN,Union Station GO,43.645195,-79.380600,2,1,0,000000,BR: Barrie; KI: Kitchener; LE: Lakeshore East;...,#003767; #00853e; #ff0d00; #98002e; #f57f25; #...,Union Station
915,WE,Weston GO,43.700220,-79.514671,4,1,0,00853e,KI: Kitchener,#00853e,KI: Kitchener
916,WH,Whitby GO,43.864840,-78.938180,93,1,0,ff0d00,LE: Lakeshore East,#ff0d00,LE: Lakeshore East


In [17]:
# Aggregate stop counts by zone
stops_by_zone = (
    stops_final[["zone_id", "stop_id", "Train", "Bus"]]
    .groupby("zone_id")
    .agg(total_stops=("stop_id", "count"),
         train_stops=("Train", "sum"),
         bus_stops=("Bus", "sum"))
    .reset_index()
    # .astype(str)
)

# Merge trips and routes
trips_routes = (
    pd.merge(trips, routes[["route_id", "route_short_name", "route_long_name", "route_color"]],
             on="route_id", how="inner")
    .drop_duplicates(subset=["shape_id", "trip_headsign", "route_id"])
    .assign(route_color=lambda df: "#" + df["route_color"],
            route_full_name=lambda df: df["route_short_name"] + ": " + df["route_long_name"])
    [["shape_id", "trip_headsign", "route_short_name", "route_long_name", "route_full_name", "route_color"]]
)

# Sort shapes by sequence and merge with route info
grouped_shapes = (
    shapes.sort_values(["shape_id", "shape_pt_sequence"])
    .merge(trips_routes, on="shape_id", how="inner")
)

In [18]:
# Load the GeoJSON file containing all polygons coordinates
geojson = folium.GeoJson(os.path.join(config["data_GIS_folder"], 'go_transit_voronoi_zones.geojson'),
                         highlight_function = lambda feat: {'fillColor': 'orange'})

# Access the GeoJson features and add counts
for feature in geojson.data['features']:

    zone_id = feature['properties']['zone_id']
    matching_rows = stops_by_zone.loc[
        stops_by_zone['zone_id'] == zone_id
    ]

    if not matching_rows.empty:
        feature['properties'].update({
            'total_stops': int(matching_rows['total_stops'].iloc[0]),
            'train_stops': int(matching_rows['train_stops'].iloc[0]),
            'bus_stops': int(matching_rows['bus_stops'].iloc[0]),
        })
    else:
        feature['properties'].update({
            'total_stops': 0,
            'train_stops': 0,
            'bus_stops': 0,
        })
        print(f"There are no stops in Zone {zone_id}.")

In [19]:
# Build route mapping
route_mapping = {
    shape_id: grouped_shapes[grouped_shapes['shape_id'] == shape_id]['route_full_name'].iloc[0]
    for shape_id in grouped_shapes['shape_id'].unique()
}

In [20]:
# Define Utility Functions
def add_polygon_layer(fg, geojson, m):
    geojson_layer = folium.GeoJson(
        geojson.data,
        name="Zones",
        tooltip=folium.GeoJsonTooltip(
            fields=["zone_id", "total_stops", "train_stops", "bus_stops"],
            aliases=["Zone ID", "Total Stops", "Train Stops", "Bus Stops"],
            localize=True,
            sticky=True,
            labels=True,
            style=(
                "background-color: white; color: black; "
                "font-family: Arial; font-size: 12px; padding: 5px;"
            )
        ),
        highlight_function=lambda feature: {
            'fillColor': '#FFA500',
            'fillOpacity': 0.3
        }
    )
    geojson_layer.add_to(fg)

    # Add zone labels at centroids
    for feature in geojson.data['features']:
        geometry = feature['geometry']
        properties = feature['properties']

        if geometry['type'] == 'Polygon':
            coordinates = geometry['coordinates'][0]
            centroid = [sum(p[1] for p in coordinates) / len(coordinates),
                        sum(p[0] for p in coordinates) / len(coordinates)]

            folium.Marker(
                location=centroid,
                icon=folium.DivIcon(html=f"""<div style="font-family: Arial; font-weight: bold; color: black">
                                                {properties['zone_id']}</div>""")
            ).add_to(fg)

def add_polyline_layers(fg, shape_data, route_mapping):
    for shape_id, shape_df in shape_data.groupby('shape_id'):
        is_train = len(str(shape_id)) == 4
        weight = 5 if is_train else 1

        route_color = shape_df['route_color'].iloc[0]
        if not route_color.startswith('#'):
            route_color = '#' + route_color

        route_name = route_mapping.get(shape_id, f"Route {shape_id}")

        # HTML tooltip with color-coded route name
        tooltip_html = f"""
        <div style="font-family: Arial; font-size: 12px">
            <div><strong>Route:</strong></div>
            <div style="color:{route_color}">{route_name}</div>
        </div>
        """

        folium.PolyLine(
            locations=shape_df[['shape_pt_lat', 'shape_pt_lon']].values,
            color=route_color,
            weight=weight,
            opacity=0.7,
            tooltip=folium.Tooltip(tooltip_html, sticky=True)
        ).add_to(fg)

def add_circle_layers(fg, stops_subset):
    for _, stop in stops_subset.iterrows():
        radius = 200 if stop['Train'] == 1 else 20
        color = f"#{stop['stop_color']}"
        
        # Split route names and colors
        route_names = stop['route_full_name'].split('; ')
        route_colors = stop['route_color'].split('; ')
        
        # Create colored lines
        colored_lines = ''.join(
            f'<div style="color:{color}">{name}</div>'
            for name, color in zip(route_names, route_colors)
        )

        # HTML tooltip
        tooltip_html = f"""
        <div style="font-family: Arial; font-size: 12px">
            <div><strong>Stop Name:</strong></div>
            {stop['stop_name']} ({'Rail' if stop['Train'] == 1 else 'Bus'})
            <div><strong>Route(s):</strong></div>
            {colored_lines}
        </div>
        """

        folium.Circle(
            location=[stop['stop_lat'], stop['stop_lon']],
            radius=radius,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.5,
            tooltip=folium.Tooltip(tooltip_html, sticky=True)
        ).add_to(fg)



# Main Map Rendering
def main():
    m = folium.Map(location=[43.645195, -79.3806], zoom_start=9, tiles=None)

    # Base Layers
    osm = folium.TileLayer(tiles='OpenStreetMap', name='OpenStreetMap', control=False).add_to(m)
    sat = folium.TileLayer(
        tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr=f"Map data ©{datetime.now().year} Google",
        name='Google Satellite',
        control=False
    ).add_to(m)
    carto = folium.TileLayer(tiles='cartodbpositron', name='CartoDB Positron', control=False).add_to(m)

    base_tree = {
        "label": "Base Maps",
        "children": [
            {"label": "OpenStreetMap", "layer": osm},
            {"label": "Google Satellite", "layer": sat},
            {"label": "CartoDB Positron", "layer": carto},
        ]
    }

    # Add Utility Controls
    plugins.MousePosition().add_to(m)
    plugins.Fullscreen(position="topleft", title="Expand Fullscreen", title_cancel="Exit Fullscreen", force_separate_button=True).add_to(m)

    # Zones
    zones_fg = folium.FeatureGroup(name="Zones", show=True).add_to(m)
    add_polygon_layer(zones_fg, geojson, m)

    # Separate rail vs. bus
    rail_routes = grouped_shapes[grouped_shapes['shape_id'].astype(str).str.len() == 4]
    bus_routes = grouped_shapes[grouped_shapes['shape_id'].astype(str).str.len() != 4]

    # Create per-route feature groups
    rail_layers = {}
    bus_layers = {}

    for route_name in sorted(rail_routes['route_full_name'].unique()):
        fg = folium.FeatureGroup(name=route_name, show=False)
        fg.add_to(m)
        rail_layers[route_name] = fg

    for route_name in sorted(bus_routes['route_full_name'].unique()):
        fg = folium.FeatureGroup(name=route_name, show=False)
        fg.add_to(m)
        bus_layers[route_name] = fg

    # Add shapes to corresponding feature groups
    def add_polyline_by_route(shape_data, layer_dict):
        for route_name, group in shape_data.groupby('route_full_name'):
            fg = layer_dict.get(route_name)
            if fg:
                add_polyline_layers(fg, group, route_mapping)

    add_polyline_by_route(rail_routes, rail_layers)
    add_polyline_by_route(bus_routes, bus_layers)

    # === Stops Layer Logic Starts Here ===

    # Create FeatureGroups for each rail_stop_label under Train == 1
    rail_stop_groups = {}
    rail_stops = stops_final[(stops_final['Train'] == 1) & (stops_final['rail_stop_label'] != 'Bus')]

    for label in sorted(rail_stops['rail_stop_label'].unique()):
        fg = folium.FeatureGroup(name=label, show=True).add_to(m)
        rail_stop_groups[label] = fg
        add_circle_layers(fg, rail_stops[rail_stops['rail_stop_label'] == label])

    # Bus stops (rail_stop_label == 'Bus')
    bus_stops_fg = folium.FeatureGroup(name="Bus Stops", show=True).add_to(m)
    bus_stops = stops_final[stops_final['rail_stop_label'] == 'Bus']
    add_circle_layers(bus_stops_fg, bus_stops)

    # Tree structure for Stops
    stops_tree = {
        "label": "Stops",
        "select_all_checkbox": True,
        "collapsed": True,
        "children": [
            {
                "label": "Rail",
                "select_all_checkbox": True,
                "collapsed": True,
                "children": [
                    {"label": label, "layer": fg}
                    for label, fg in rail_stop_groups.items()
                ]
            },
            {
                "label": "Bus",  # invisible parent node
                "select_all_checkbox": True,
                "collapsed": True,
                "children": [
                    {"label": "All Bus Stops", "layer": bus_stops_fg}
                ]
            }
        ]
    }

    # === Overlay Tree with New Stops Section ===
    overlay_tree = {
        "label": "GO Transit",
        "select_all_checkbox": True,
        "collapsed": False,
        "children": [
            {
                "label": "Zones (Voronoi Polygons)",
                "select_all_checkbox": True,
                "collapsed": True,
                "children": [
                    {"label": "All Zones", "layer": zones_fg}
                ]
            },
            {
                "label": "Routes",
                "select_all_checkbox": True,
                "collapsed": True,
                "children": [
                    {
                        "label": "Rail",
                        "select_all_checkbox": True,
                        "collapsed": True,
                        "children": [
                            {"label": name, "layer": fg}
                            for name, fg in rail_layers.items()
                        ]
                    },
                    {
                        "label": "Bus",
                        "select_all_checkbox": True,
                        "collapsed": True,
                        "children": [
                            {"label": name, "layer": fg}
                            for name, fg in bus_layers.items()
                        ]
                    }
                ]
            },
            stops_tree
        ]
    }

    # Add Tree Layer Control
    plugins.TreeLayerControl(
        base_tree=base_tree,
        overlay_tree=overlay_tree,
        named_toggle=False,
        collapse_all="Collapse all",
        expand_all="Expand all"
    ).add_to(m)

    # display(m)
    # Add fixed disclaimer panel with a brief method summary
    disclaimer = """
    {% macro html(this, kwargs) %}
    <div style="
        position: fixed;
        bottom: 10px;
        left: 10px;
        z-index: 9999;
        background: rgba(255,255,255,0.95);
        padding: 8px 10px;
        border: 1px solid #bbb;
        border-radius: 6px;
        box-shadow: 0 1px 4px rgba(0,0,0,0.25);
        max-width: 380px;
        font-size: 12px;
        line-height: 1.35;">
        
        <div style="font-weight:600; margin-bottom:4px;">Disclaimer</div>
        
        <div>This is an independent, personal project and does not represent official GO Transit fare zones, designs, or policies. All views are solely my own.</div>
        
        <div style="margin-top:6px;">
            The zones shown are derived from publicly available <a href="https://www.gotransit.com/en/partner-with-us/software-developers" target="_blank" rel="noopener">GO GTFS</a> using Voronoi diagram in a projected coordinate reference system. The polygons were clipped to a custom-generated service boundary, designed to ensure all GO Transit stops and routes fall within it, and dissolved by zone identifier. Finally, the zones were reprojected to WGS84 for web visualization.
        </div>
        
        <div style="margin-top:6px;">
            Source code and methodology: <a href="https://github.com/zhenhuasun/go-zone-network-explorer/" target="_blank" rel="noopener">github.com/zhenhuasun/go-zone-network-explorer/</a>
        </div>
    </div>
    {% endmacro %}
    """
    macro = MacroElement()
    macro._template = Template(disclaimer)
    m.get_root().add_child(macro)

    # Save the map
    m.save(os.path.join(config["outputs_folder"], 
                    "02_go_transit_voronoi_fare_zones_treelayer.html"))

main()